# Lab Assignment Six: Wide and Deep Networks



__*Austin Chen, Luke Hansen, Oscar Vallner*__

## 1. Preparation and Overview


### 1.1 Business Understanding

For the 6th Lab Assignment, we will be using the "Diabetes 130-US hospitals of years 1999-2008" dataset, downloaded from the UCI Machine Learning Repository. This dataset has been prepared for usage in analyzing medical factors related to hospital readmission for patients with diabetes. The dataset represents ten years worth of clinical care data from 130 hospitals in the United States, as well as integrated delivery networks. In order for an entry to qualify for the dataset, it must satisfy the following requirements:

1. It must be a hospital admission
2. The admission must be diabetic related
3. The length of the stay must be more than 1 day but less than 2 weeks.
4. Laboratory tests must have been performed on the patient
5. Medications must have been administered or prescribed.

The dataset's features comprise of a mix of nominal and numeric datatypes. The features span a multitude of patient information categories such as race, gender, age, weight, and time spent in hospital. Some of the more specific features include information such as medications administered (miglitol, metformin, tolazamide, etc). 

The features and content of this dataset can provide extremely important information for the healthcare industry, for various reasons. First and foremost, a classification model that can accurately predict whether a diabetic inpatient will return to the hospital has several implications in itself. On the medical side, knowing that a patient is at risk for rehospitalization, given a patient's specific attributes, can aid doctors and physicians modify their treatment methodology to help minimize the chance of rehospitalization. A similar concept applies for the patients themselves; patients who can know that they may be at risk for rehospitalization can take extra precautions once they are discharged, in an attempt to prevent further emergencies. And finally, hospital administrators could use this risk information in order to better plan room reservations, medical supply projections, and logisitc planning if they know that patients will be readmitted. Though it is difficult to plan resources for hospitals due to their hectic nature, possessing the information to help plan is still there nonetheless. In any case, the business applications are wide and deep no matter the stakeholder.

With the aforementioned data, we plan to construct a classifier that will predict whether or not a patient will be rehospitalized within 30 days after discharge.

An important issue regarding classification must be highlighted, however. The original dataset creates three distinct class boundaries for hospital readmittance:

1. NO: No record of readmission
2. &lt;30: If the patient was readmitted in less than 30 days.
3. &gt;30: If the patient was readmitted in more than 30 days.

For the sake of our business case, we believe it would be far more feasible to narrow the scope to a binary classification. Instead of the three distinct classes seen above, a simplification could prove more pratical and informative:

1. NO: No record of readmission
2. &lt;30: If the patient was readmitted in less than 30 days.

We have decided to eliminate the third type above (>30 days) for a number of reasons. First, it is important to reiterate that one of our primary business cases is to help medical professionals improve their treatment methodologies on diabetic hospital patients. Once a patient has been released for more than 30 days, the more difficult it is to blame any rehospitalizations on the aspects of the procedure itself. There could be several extenuating circumstances beyond the hospital treatment that contribute to a rehospitalization of a patient. Furthermore, limiting the scope to a binary classification can greatly enhance our ability to choose the most appropriate evaluation method. For an industry as important as healthcare, especially where human life is at risk, we believe that a simpler classification, with better evaluation practices and better performance, is more deployable than a multi-class problem with lower performance. With more time, we can continue to refine our classification model to accomodate more classes. With regards to our specific dataset, we would require consultation with a domain expert in order to evaluate the extenuating circumstances that contribute to rehospitalization past 30 days.

---

Link to dataset: http://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

---